In [1]:
#! pip install -U climetlab --quiet
#! pip install -U climetlab_s2s_ai_challenge --quiet

In [2]:
import climetlab as cml 

In [3]:
import climetlab_s2s_ai_challenge
print(f'Climetlab version : {cml.__version__}')
print(f'Climetlab-s2s-ai-challenge plugin version : {climetlab_s2s_ai_challenge.__version__}')

Climetlab version : 0.7.1
Climetlab-s2s-ai-challenge plugin version : 0.6.7


# Observations data for training

Climetlab provides the observation datasets. They can be used as a xarray.Dataset :

In [4]:
cmlds = cml.load_dataset("s2s-ai-challenge-training-output-reference",
                         date = 20200102,
                         parameter='t2m'                         
                       )
cmlds.to_xarray()

By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/data/s2s/licence/. If you do not agree with such terms, do not download the data.  This dataset has been dowloaded from IRIDL. By downloading this data you also agree to the terms and conditions defined at https://iridl.ldeo.columbia.edu.


  0%|          | 0/1 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:        (forecast_time: 20, latitude: 121, lead_time: 47, longitude: 240)
Coordinates:
    valid_time     (lead_time, forecast_time) datetime64[ns] dask.array<chunksize=(47, 20), meta=np.ndarray>
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * forecast_time  (forecast_time) datetime64[ns] 2000-01-02 ... 2019-01-02
  * lead_time      (lead_time) timedelta64[ns] 0 days 1 days ... 45 days 46 days
Data variables:
    t2m            (lead_time, forecast_time, latitude, longitude) float32 dask.array<chunksize=(47, 20, 121, 240), meta=np.ndarray>
Attributes:
    source_dataset_name:  temperature daily from NOAA NCEP CPC: Climate Predi...
    source_hosting:       IRIDL
    source_url:           http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/...
    created_by_software:  climetlab-s2s-ai-challenge
    created_by_script:    tools/observations/makefile

In [5]:
cmlds = cml.load_dataset("s2s-ai-challenge-training-output-reference",
                         date = 20200102,
                         parameter='tp'                         
                       )
cmlds.to_xarray()

  0%|          | 0/1 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:        (forecast_time: 20, latitude: 121, lead_time: 47, longitude: 240)
Coordinates:
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * forecast_time  (forecast_time) datetime64[ns] 2000-01-02 ... 2019-01-02
  * lead_time      (lead_time) timedelta64[ns] 0 days 1 days ... 45 days 46 days
    valid_time     (lead_time, forecast_time) datetime64[ns] dask.array<chunksize=(47, 20), meta=np.ndarray>
Data variables:
    tp             (lead_time, forecast_time, latitude, longitude) float32 dask.array<chunksize=(47, 20, 121, 240), meta=np.ndarray>
Attributes:
    source_dataset_name:  NOAA NCEP CPC UNIFIED_PRCP GAUGE_BASED GLOBAL v1p0 ...
    source_hosting:       IRIDL
    source_url:           http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/...
    created_by_software:  climetlab-s2s-ai-challenge
    created_by_script:    tools/observations/makefile